In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
import nltk 
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Navaneeth Gowda
[nltk_data]     T\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [27]:
df=pd.read_csv("Tweets.csv", encoding='ISO-8859-1')

In [28]:
df.shape

(14640, 15)

In [29]:
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [20]:
column_names=['text', 'target']
df=pd.read_csv("Twitter_Data.csv", names=column_names, encoding='ISO-8859-1')

In [30]:
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [31]:
df.isnull().sum()

tweet_id                            0
airline_sentiment                   0
airline_sentiment_confidence        0
negativereason                   5462
negativereason_confidence        4118
airline                             0
airline_sentiment_gold          14600
name                                0
negativereason_gold             14608
retweet_count                       0
text                                0
tweet_coord                     13621
tweet_created                       0
tweet_location                   4733
user_timezone                    4820
dtype: int64

In [32]:
newdf=df.dropna()

In [33]:
newdf.isnull().sum()

tweet_id                        0
airline_sentiment               0
airline_sentiment_confidence    0
negativereason                  0
negativereason_confidence       0
airline                         0
airline_sentiment_gold          0
name                            0
negativereason_gold             0
retweet_count                   0
text                            0
tweet_coord                     0
tweet_created                   0
tweet_location                  0
user_timezone                   0
dtype: int64

In [34]:
newdf['airline_sentiment'].value_counts()

airline_sentiment
negative    2
Name: count, dtype: int64

In [35]:
port_stem=PorterStemmer()

In [36]:
def stemming(content):
    stemmed_content=re.sub('[^a-zA-Z]',' ', content)
    stemmed_content=stemmed_content.lower()
    stemmed_content=stemmed_content.split()
    stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content=' '.join(stemmed_content)
    
    return stemmed_content


In [42]:
df['stemmed_content']=df['text'].apply(stemming)

In [46]:
print(df['airline_sentiment'])

0         neutral
1        positive
2         neutral
3        negative
4        negative
           ...   
14635    positive
14636    negative
14637     neutral
14638    negative
14639     neutral
Name: airline_sentiment, Length: 14640, dtype: object


In [47]:
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,stemmed_content
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada),virginamerica dhepburn said
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada),virginamerica plu ad commerci experi tacki
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada),virginamerica today must mean need take anoth ...
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada),virginamerica realli aggress blast obnoxi ente...
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada),virginamerica realli big bad thing


In [48]:
print(df['stemmed_content'])

0                              virginamerica dhepburn said
1               virginamerica plu ad commerci experi tacki
2        virginamerica today must mean need take anoth ...
3        virginamerica realli aggress blast obnoxi ente...
4                       virginamerica realli big bad thing
                               ...                        
14635          americanair thank got differ flight chicago
14636    americanair leav minut late flight warn commun...
14637    americanair pleas bring american airlin blackb...
14638    americanair money chang flight answer phone su...
14639    americanair ppl need know mani seat next fligh...
Name: stemmed_content, Length: 14640, dtype: object


In [49]:
#separating the data and label

X=df['stemmed_content'].values
y=df['airline_sentiment'].values

In [51]:
print(X)

['virginamerica dhepburn said'
 'virginamerica plu ad commerci experi tacki'
 'virginamerica today must mean need take anoth trip' ...
 'americanair pleas bring american airlin blackberri'
 'americanair money chang flight answer phone suggest make commit'
 'americanair ppl need know mani seat next flight plz put us standbi peopl next flight']


In [52]:
print(y)

['neutral' 'positive' 'neutral' ... 'neutral' 'negative' 'neutral']


In [54]:
#Splitting the data to training Data and test Data

X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [55]:
print(X.shape, X_train.shape, X_test.shape)

(14640,) (11712,) (2928,)


In [56]:
print(X_train)

['unit would cost'
 'usairway use get email pre purchas snack time check got neither tomorrow trip get sent'
 'unit flight cancel flightlat one due weather one mechan paid hotel bag held transfer voucher compens'
 ...
 'southwestair assign seat cousin amp probabl sit togeth flight tomorrow thank'
 'jetblu enough stop fli jetblu'
 'southwestair beauti fleet perfect even fli http co xmz tf ix']


In [57]:
print(X_test)

['unit past' 'jetblu would say delay like thank much'
 'jetblu cheat sorri never southwestair given wife worst start honeymoon ever'
 ...
 'southwestair next arriv atlanta wait full hour luggag place carousel one hour'
 'unit look like today th consecut delay flight win prize southwest air fli sba'
 'southwestair flight cancel flightl monday look cancel flight credit card']


In [59]:
#converting the text data to numerical data
vectorizer=TfidfVectorizer()

X_train=vectorizer.fit_transform(X_train)
X_test=vectorizer.transform(X_test)

In [60]:
print(X_train)

  (0, 1684)	0.7851030491414758
  (0, 9155)	0.5444467042337754
  (0, 8541)	0.29528120237789857
  (1, 7226)	0.25354172985917084
  (1, 8324)	0.24477856870139028
  (1, 8230)	0.23196087380060265
  (1, 5487)	0.3937062834482747
  (1, 3348)	0.21147026671128047
  (1, 1326)	0.21083441564259245
  (1, 8165)	0.17912723780244574
  (1, 7485)	0.34565790522235706
  (1, 6485)	0.3076230672586847
  (1, 6340)	0.3342033428688576
  (1, 2457)	0.23376860271873365
  (1, 3230)	0.3090829470654168
  (1, 8666)	0.23397377524288165
  (1, 8649)	0.12249570790975114
  (2, 1564)	0.2940934706361922
  (2, 8830)	0.2601753392812513
  (2, 8283)	0.31634374117699215
  (2, 3590)	0.3224633329013843
  (2, 602)	0.18444165351557543
  (2, 3750)	0.25317089252920383
  (2, 5997)	0.27917973245455463
  (2, 5086)	0.28386705568511245
  :	:
  (11709, 8214)	0.3459379528218551
  (11709, 455)	0.3889851451746868
  (11709, 6404)	0.38517101097944245
  (11709, 7186)	0.24888340647144463
  (11709, 287)	0.22985874961603026
  (11709, 7384)	0.2750234571

In [61]:
print(X_test)

  (0, 8541)	0.3480862282137573
  (0, 6049)	0.9374625207057187
  (1, 9155)	0.39004048547213804
  (1, 8042)	0.2873991683893263
  (1, 7114)	0.43797293709361607
  (1, 5372)	0.47251401025039713
  (1, 4716)	0.40828544079435003
  (1, 4248)	0.26178461885295967
  (1, 1975)	0.33904596496616124
  (2, 9138)	0.2612058828876238
  (2, 9044)	0.3104451757218134
  (2, 7671)	0.28772528620874943
  (2, 7566)	0.14231598416312588
  (2, 7547)	0.3025052585810406
  (2, 5503)	0.2438655930093064
  (2, 4248)	0.14658286551998256
  (2, 3712)	0.4008351992008632
  (2, 3272)	0.33034973011541485
  (2, 2609)	0.26066497865484933
  (2, 1324)	0.47132066828631153
  (3, 8635)	0.2992165114158297
  (3, 8541)	0.10926623442112508
  (3, 7933)	0.21941684643550824
  (3, 5246)	0.3458030309845443
  (3, 3714)	0.34351462435935265
  :	:
  (2925, 3128)	0.3143701708876917
  (2925, 1185)	0.42545703360860443
  (2925, 478)	0.3627689565983508
  (2925, 427)	0.2786232840054751
  (2926, 9057)	0.3471962794769463
  (2926, 8541)	0.11114326481320809


In [62]:
#training the ML model
model=LogisticRegression(max_iter=1000)

In [64]:
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [66]:
X_train_predict=model.predict(X_train)
accuracy=accuracy_score(y_train, X_train_predict)

In [67]:
accuracy

0.8685963114754098

In [70]:
X_test_predict=model.predict(X_test)
accuracy1=accuracy_score(y_test, X_test_predict)

In [71]:
accuracy

0.789275956284153

In [72]:
def predict_sentiment(user_input):
    # Preprocess the user input
    stemmed_input = stemming(user_input)
    # Convert the preprocessed input to a format compatible with the model
    processed_input = vectorizer.transform([stemmed_input])
    # Use the model to predict the sentiment of the user input
    sentiment_prediction = model.predict(processed_input)
    return sentiment_prediction[0]

In [166]:
user_input = input("Please enter a sentence: ")
print(predict_sentiment(user_input))

Please enter a sentence:  terrific


negative
